# 의사결정나무 (Decision Tree)

- 의사결정을 위한 규칙을 나무 모양으로 조합하여, 목표변수에 대한 분류를 수행하는 기법이다.
- 수치 데이터, 범주 데이터 모두 사용 가능하다.

## package import

In [243]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import matplotlib.pyplot as plt
import scipy.stats as stats

import sklearn

In [244]:
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"sklearn version: {sklearn.__version__}")

pandas version: 2.2.2
numpy version: 1.26.4
matplotlib version: 3.9.0
scipy version: 1.13.1
sklearn version: 1.5.0


## 데이터 불러오기

In [246]:
df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/titanic.csv")

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [247]:
print(df.info())
print()
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.4865

## 데이터 전처리

In [248]:
# Age 결측치 대체 (평균값)
df["Age"] = df["Age"].fillna(df["Age"].mean())

assert df["Age"].isna().sum() == 0

In [249]:
# Embarked 결측치 대체 (최빈값)

df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

assert df["Embarked"].isna().sum() == 0

In [250]:
# Sex Label Encoding 

from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

labelEncoder.fit(sorted(np.unique(df["Sex"])))
df["Sex"] = labelEncoder.transform(df["Sex"])

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.000000,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.000000,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,29.699118,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",1,26.000000,0,0,111369,30.0000,C148,C


In [251]:
# Embarked Label Encoding

from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

df["Embarked"] = labelEncoder.fit_transform(df["Embarked"])

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.000000,0,0,211536,13.0000,NaN,2
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.000000,0,0,112053,30.0000,B42,2
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,29.699118,1,2,W./C. 6607,23.4500,NaN,2
889,890,1,1,"Behr, Mr. Karl Howell",1,26.000000,0,0,111369,30.0000,C148,0


In [252]:
# SibSp, Parch 합쳐서 FamilySize

df["FamilySize"] = df["SibSp"] + df["Parch"]

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,2,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,0,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,2,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,2,1
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.000000,0,0,211536,13.0000,NaN,2,0
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.000000,0,0,112053,30.0000,B42,2,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,29.699118,1,2,W./C. 6607,23.4500,NaN,2,3
889,890,1,1,"Behr, Mr. Karl Howell",1,26.000000,0,0,111369,30.0000,C148,0,0


## train, test 데이터 준비

In [253]:
# train, test 
from sklearn.model_selection import train_test_split

X = df[["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize"]]
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(712, 6)
(179, 6)
(712,)
(179,)


## 모델 적용 및 데이터 분석 수행

```python
sklearn.tree.DecisionTreeClassifier(
    *,
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0,
    monotonic_cst=None,
)
```

In [260]:
from sklearn.tree import DecisionTreeClassifier

decisionTree = DecisionTreeClassifier()

decisionTree.fit(X_train, y_train)

y_pred = decisionTree.predict(X_test)

assert y_pred.shape == y_test.shape

## 데이터 모델링 성능 평가

In [259]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

report = classification_report(y_test, y_pred)

print(report)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred, average=None))
print(recall_score(y_test, y_pred, average=None))
print(f1_score(y_test, y_pred, average=None))

print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84       118
           1       0.69      0.72      0.70        61

    accuracy                           0.79       179
   macro avg       0.77      0.78      0.77       179
weighted avg       0.80      0.79      0.79       179

0.7932960893854749
[0.85217391 0.6875    ]
[0.83050847 0.72131148]
[0.84120172 0.704     ]
[[98 20]
 [17 44]]


# KNN (K-Nearest Neighbor)

- KNN 알고리즘은 변수별 단위가 무엇이냐에 따라 거리가 다라지고, 분류 결과가 달라질 수 있다.
- 따라서, KNN 알고리즘을 적용할 때에는 사전에 데이터를 표준화 해야한다.

## package import

In [185]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

## 데이터 불러오기

In [186]:
# data load
df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/iris.csv")
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [192]:
len(df.groupby("species"))

len(np.unique(df["species"]))

3

In [193]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


## 데이터 전처리

In [210]:
# Min-Max Normalization

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df2 = df.drop("species", axis=1)

df = df.join(pd.DataFrame(scaler.fit_transform(df2), columns=df2.columns + "_norm"))

df

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length_norm,sepal_width_norm,petal_length_norm,petal_width_norm
0,5.1,3.5,1.4,0.2,setosa,0.222222,0.625000,0.067797,0.041667
1,4.9,3.0,1.4,0.2,setosa,0.166667,0.416667,0.067797,0.041667
2,4.7,3.2,1.3,0.2,setosa,0.111111,0.500000,0.050847,0.041667
3,4.6,3.1,1.5,0.2,setosa,0.083333,0.458333,0.084746,0.041667
4,5.0,3.6,1.4,0.2,setosa,0.194444,0.666667,0.067797,0.041667
...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,0.666667,0.416667,0.711864,0.916667
146,6.3,2.5,5.0,1.9,virginica,0.555556,0.208333,0.677966,0.750000
147,6.5,3.0,5.2,2.0,virginica,0.611111,0.416667,0.711864,0.791667
148,6.2,3.4,5.4,2.3,virginica,0.527778,0.583333,0.745763,0.916667


## train, test 데이터 준비

In [219]:
# train test split

from sklearn.model_selection import train_test_split

columns = df.columns[df.columns.str.endswith("_norm")]

X = df[columns]
y = df["species"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120, 4)
(30, 4)
(120,)
(30,)


## 모델 적용 및 데이터 분석 수행

```python
sklearn.neighbors.KNeighborsClassifier(
    n_neighbors=5,
    *,
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    metric_params=None,
    n_jobs=None,
)
```

In [236]:
# KNN 모델 적용

from sklearn.neighbors import KNeighborsClassifier

# n_neighbors = len(np.unique(df["species"]))
n_neighbors = 6
knn = KNeighborsClassifier(n_neighbors=n_neighbors)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

assert y_pred.shape == y_test.shape

## 데이터 모델링 성능 평가

In [241]:
# 모델 성능 - 정확도 측정

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

report = classification_report(y_test, y_pred)

print(report)

print(f"accuracy = {accuracy_score(y_test, y_pred)}")
print(f"recall = {recall_score(y_test, y_pred, average=None)}")
print(f"precision_score = {precision_score(y_test, y_pred, average=None)}")
print(f"f1_score = {f1_score(y_test, y_pred, average=None)}")

print()
print(f"confusion_matrix = \n{confusion_matrix(y_test, y_pred)}")

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         9
  versicolor       0.83      1.00      0.91        10
   virginica       1.00      0.82      0.90        11

    accuracy                           0.93        30
   macro avg       0.94      0.94      0.94        30
weighted avg       0.94      0.93      0.93        30

accuracy = 0.9333333333333333
recall = [1.         1.         0.81818182]
precision_score = [1.         0.83333333 1.        ]
f1_score = [1.         0.90909091 0.9       ]

confusion_matrix = 
[[ 9  0  0]
 [ 0 10  0]
 [ 0  2  9]]


# SVM (Support Vector Machine)

- 비교적 적은 학습 데이터로도 정확도가 높은 분류를 기대할 수 있다.
  - 이를 위해서는 데이터의 전처리 과정을 통해 데이터의 특성을 잘 표현하도록 해야한다.
- 변수가 많은 경우 결정 경계 및 데이터의 시각화가 어려워 분류 결과의 이해가 어렵다. 

## package import

In [300]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split

from sklearn import svm

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(f"numpy version = {np.__version__}")
print(f"pandas version = {pd.__version__}")
print(f"sklearn version = {sklearn.__version__}")

numpy version = 1.26.4
pandas version = 2.2.2
sklearn version = 1.5.0


## 데이터 불러오기

In [331]:
df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [294]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [295]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 데이터 전처리

In [421]:
# Age 결측치 평균값으로 대체

df["Age"] = df["Age"].fillna(df["Age"].mean())

assert df["Age"].isna().sum() == 0

KeyError: 'Age'

In [333]:
# Embarked 결측치 최빈값으로 대체

df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

assert df["Embarked"].isna().sum() == 0

In [334]:
# Sibsp, Parch 두 값을 더해서 FamilySize 칼럼 추가

# df = df.join(pd.DataFrame({"FamilySize": df["SibSp"] + df["Parch"]}))

df["FamilySize"] = df["SibSp"] + df["Parch"]

assert (df["FamilySize"] == df["SibSp"] + df["Parch"]).all()

In [370]:
# Sex, Embarked One-Hot 인코딩

if "sex_male" not in df.columns and "sex_female" not in df.columns:
    # df = df.join(pd.get_dummies(df["Sex"], prefix="sex", prefix_sep="_", dtype="int"))
    df = pd.concat([df, pd.get_dummies(df["Sex"], prefix="sex", prefix_sep="_", dtype="int")], axis=1)

if not all(column in df.columns for column in "Embarked_" + np.unique(df["Embarked"])):
    df = df.join(pd.get_dummies(df["Embarked"], prefix="Embarked").astype(int))
    # df = pd.concat([df, pd.get_dummies(df["Embarked"], prefix="Embarked").astype(int)], axis=1)
    
assert np.isin(["sex_male", "sex_female"], df.columns).all()
assert all(column in df.columns for column in ["Embarked_C", "Embarked_Q", "Embarked_S"])

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,sex_female,sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,1,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,1,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,1,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,0,0,1,0,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,0,1,0,0,0,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,3,1,0,0,0,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,0,0,1,1,0,0


## train, test 데이터 준비

In [372]:
from sklearn.model_selection import train_test_split

X = df[["Pclass", "Age", "Fare", "FamilySize", "sex_female", "sex_male", "Embarked_C", "Embarked_Q", "Embarked_S"]]
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(623, 9)
(268, 9)
(623,)
(268,)


## 모델 적용 및 데이터 분석 수행

```python
sklearn.svm.SVC(
    *,
    C=1.0,
    kernel='rbf',
    degree=3,
    gamma='scale',
    coef0=0.0,
    shrinking=True,
    probability=False,
    tol=0.001,
    cache_size=200,
    class_weight=None,
    verbose=False,
    max_iter=-1,
    decision_function_shape='ovr',
    break_ties=False,
    random_state=None,
)
```

In [376]:
from sklearn import svm

svc = svm.SVC() # kernel="rbf"
# svc = svm.SVC(kernel="rbf")
# svc = svm.SVC(kernel="linear", C=1, gamma=0.1)
# svc = svm.SVC(kernel="rbf", C=0.1, gamma=0.1)

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

assert y_pred.shape == y_test.shape

## 데이터 모델링 성능 평가

In [383]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(f"accuracy_socre = {accuracy_score(y_test, y_pred)}")
print(f"precision_score = {precision_score(y_test, y_pred, average=None)}")
print(f"recall_score = {recall_score(y_test, y_pred, average=None)}")
print(f"f1_score = {f1_score(y_test, y_pred, average=None)}")

print()
report = classification_report(y_test, y_pred)

print(report)

print(f"confusion_matrix = \n{confusion_matrix(y_test, y_pred)}")

accuracy_socre = 0.7201492537313433
precision_score = [0.7106383  0.78787879]
recall_score = [0.95977011 0.27659574]
f1_score = [0.81662592 0.40944882]

              precision    recall  f1-score   support

           0       0.71      0.96      0.82       174
           1       0.79      0.28      0.41        94

    accuracy                           0.72       268
   macro avg       0.75      0.62      0.61       268
weighted avg       0.74      0.72      0.67       268

confusion_matrix = 
[[167   7]
 [ 68  26]]


# 로지스틱 회귀 (Logistic Regression)

- Sigmoid 함수의 출력값을 각 분류 항목에 속하게 될 확률값으로 사용
  - 이 값은 0과 1사이의 실수
- 확률에 따라 가능성이 더 높은 범주에 속하는 것으로 분류하는 이진 분류 모델이다.
- 현재 갖고있는 데이터를 통해 에러를 줄이는 방향으로
  - `weight`, `bias` 의 최적값을 찾아간다.

## package import

In [385]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(f"numpy version = {np.__version__}")
print(f"pandas version = {pd.__version__}")
print(f"sklearn version = {sklearn.__version__}")

numpy version = 1.26.4
pandas version = 2.2.2
sklearn version = 1.5.0


## 데이터 불러오기

In [386]:
df = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/iris.csv")

df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [387]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## 데이터 전처리

In [417]:
# Normalization 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_number = df.select_dtypes("number")
df_number_scaled = pd.DataFrame(scaler.fit_transform(df_number), columns=df_number.columns)

df[df_number.columns] = df_number_scaled

assert (df.describe().loc["min"] == 0.0).all()
assert (df.describe().loc["max"] == 1.0).all()

df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,0.428704,0.440556,0.467458,0.458056
std,0.230018,0.181611,0.299203,0.317599
min,0.000000,0.000000,0.000000,0.000000
25%,0.222222,0.333333,0.101695,0.083333
50%,0.416667,0.416667,0.567797,0.500000
75%,0.583333,0.541667,0.694915,0.708333
max,1.000000,1.000000,1.000000,1.000000


## train, test 데이터 준비

In [419]:
from sklearn.model_selection import train_test_split

X = df.select_dtypes(np.number)
y = df["species"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120, 4)
(30, 4)
(120,)
(30,)


## 모델 적용 및 데이터 분석 수행

```python
sklearn.linear_model.LogisticRegression(
    penalty='l2',
    *,
    dual=False,
    tol=0.0001,
    C=1.0,
    fit_intercept=True,
    intercept_scaling=1,
    class_weight=None,
    random_state=None,
    solver='lbfgs',
    max_iter=100,
    multi_class='deprecated',
    verbose=0,
    warm_start=False,
    n_jobs=None,
    l1_ratio=None,
)
```

In [426]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

assert y_pred.shape == y_test.shape

## 데이터 모델링 성능 평가

In [433]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(f"accuracy_score = {accuracy_score(y_test, y_pred)}")
print(f"recall_score = {recall_score(y_test, y_pred, average=None)}")
print(f"precision_score = {precision_score(y_test, y_pred, average=None)}")
print(f"f1_score = {f1_score(y_test, y_pred, average=None)}")
print()

report = classification_report(y_test, y_pred)

print(report)

print(f"confusion_matrix = \n {confusion_matrix(y_test, y_pred)}")

accuracy_score = 0.8333333333333334
recall_score = [1.         0.7        0.81818182]
precision_score = [1.         0.77777778 0.75      ]
f1_score = [1.         0.73684211 0.7826087 ]

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         9
  versicolor       0.78      0.70      0.74        10
   virginica       0.75      0.82      0.78        11

    accuracy                           0.83        30
   macro avg       0.84      0.84      0.84        30
weighted avg       0.83      0.83      0.83        30

confusion_matrix = 
 [[9 0 0]
 [0 7 3]
 [0 2 9]]


# 랜덤 포레스트 (Random Forest)

## package import

## 데이터 불러오기

## 데이터 전처리

## train, test 데이터 준비

## 모델 적용 및 데이터 분석 수행

## 데이터 모델링 성능 평가